# Example Model Servers with Seldon

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](seldon_core_setup.ipynb#Ambassador) and [Seldon Core](seldon_core_setup.ipynb#Install-Seldon-Core). Instructions [also online](./seldon_core_setup.html).

## Serve SKlearn Iris Model
We can deploy a sklearn model uploaded to an object store by using the sklearn model server implementation as the config below:

In [17]:
%%writefile ../servers/sklearnserver/samples/iris.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: sklearn
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    name: default
    replicas: 1
    svcOrchSpec: 
      env: 
      - name: SELDON_LOG_LEVEL
        value: DEBUG

Overwriting ../servers/sklearnserver/samples/iris.yaml


And then we apply it to deploy it to our kubernetes cluster

In [18]:
!kubectl apply -f ../servers/sklearnserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io/sklearn created


In [13]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn -o jsonpath='{.items[0].metadata.name}')

deployment "sklearn-default-0-classifier" successfully rolled out


Once it's deployed we can send our sklearn model requests

In [14]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="sklearn",namespace="seldon")

In [15]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,4))
print(r)
assert(r.success==True)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 4
    values: 0.8842453142543456
    values: 0.2433805830234711
    values: 0.6450701414816686
    values: 0.987171241902928
  }
}

Response:
{'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.12891198935095496, 0.3592718583399204, 0.5118161523091246]}}, 'meta': {}}


And delete the model we deployed

In [16]:
!kubectl delete -f ../servers/sklearnserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io "sklearn" deleted


## Serve XGBoost Iris Model
We can deploy a xgboost model uploaded to an object store by using the xgboost model server implementation as the config below:

In [ ]:
%%writefile ../servers/xgboostserver/samples/iris.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: xgboost
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: XGBOOST_SERVER
      modelUri: gs://seldon-models/xgboost/iris
      name: classifier
    name: default
    replicas: 1

And then we apply it to deploy it to our kubernetes cluster

In [ ]:
!kubectl apply -f ../servers/xgboostserver/samples/iris.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=xgboost -o jsonpath='{.items[0].metadata.name}')

Once it's deployed we can send our xgboost model requests

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="xgboost",namespace="seldon")

In [ ]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,4))
print(r)
assert(r.success==True)

And delete the model we deployed

In [ ]:
!kubectl delete -f ../servers/xgboostserver/samples/iris.yaml

## Serve Tensorflow MNIST Model
We can deploy a tensorflow model uploaded to an object store by using the tensorflow model server implementation as the config below.

This notebook contains two examples, one which shows how you can use the TFServing prepackaged serve with the Seldon Protocol, and a second one which shows how you can deploy it using the tensorlfow protocol (so you can send requests of the exact format as you would to a tfserving server).

### Serve Tensorflow MNIST Model with Seldon Protocol

The config file below shows how you can deploy your Tensorflow model which exposes the Seldon protocol.

In [19]:
%%writefile ../servers/tfserving/samples/mnist_rest.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: tfserving
spec:
  name: mnist
  predictors:
  - graph:
      children: []
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/mnist-model
      name: mnist-model
      parameters:
        - name: signature_name
          type: STRING
          value: predict_images
        - name: model_name
          type: STRING
          value: mnist-model
    name: default
    replicas: 1

Overwriting ../servers/tfserving/samples/mnist_rest.yaml


Once it's deployed we can send our sklearn model requests

In [20]:
!kubectl apply -f ../servers/tfserving/samples/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io/tfserving created


In [21]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=tfserving -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "tfserving-default-0-mnist-model" rollout to finish: 0 of 1 updated replicas are available...
deployment "tfserving-default-0-mnist-model" successfully rolled out


Once it's deployed we can send our sklearn model requests

In [22]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="tfserving",namespace="seldon")

In [23]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,784))
print(r)
assert(r.success==True)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 784
    values: 0.39534222087132653
    values: 0.8955735466832198
    values: 0.5078445910618445
    values: 0.19081311029518344
    values: 0.0842658824002831
    values: 0.7131888883723178
    values: 0.2633686826637739
    values: 0.4279869816054471
    values: 0.5073735866047083
    values: 0.6465617552836288
    values: 0.35225995168548896
    values: 0.477886271049599
    values: 0.12280547601492242
    values: 0.009267299227261772
    values: 0.36930856387795286
    values: 0.6537949834500131
    values: 0.489890872755134
    values: 0.7778249406591495
    values: 0.8389474751076914
    values: 0.6696160378096019
    values: 0.1509523507448225
    values: 0.3883224568580801
    values: 0.0322083300247098
    values: 0.75633064803919
    values: 0.09638442222876531
    values: 0.6458316161907406
    values: 0.02436470379194644
    values: 0.13062450697994787
    values: 0.7765464550653075
    value

And delete the model we deployed

In [ ]:
!kubectl delete -f ../servers/tfserving/samples/mnist_rest.yaml

## Serve Tensorflow Model with Tensorflow protocol

The config file below shows how you can deploy your Tensorflow model which exposes the Tensorflow protocol.

In [8]:
%%writefile ../servers/tfserving/samples/halfplustwo_rest.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: hpt
spec:
  name: hpt
  protocol: tensorflow
  transport: rest
  predictors:
  - graph:
      children: []
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/half_plus_two
      name:  halfplustwo
      parameters:
        - name: model_name
          type: STRING
          value: halfplustwo
    name: default
    replicas: 1

Overwriting ../servers/tfserving/samples/halfplustwo_rest.yaml


In [ ]:
!kubectl apply -f ../servers/tfserving/samples/halfplustwo_rest.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=hpt -o jsonpath='{.items[0].metadata.name}')

In [ ]:
import json
X=!curl -s -d '{"instances": [1.0, 2.0, 5.0]}' \
   -X POST http://localhost:8003/seldon/seldon/hpt/v1/models/halfplustwo/:predict \
   -H "Content-Type: application/json"
d=json.loads("".join(X))
print(d)
assert(d["predictions"][0] == 2.5)

In [ ]:
!kubectl delete -f ../servers/tfserving/samples/halfplustwo_rest.yaml

## Serve MLFlow Elasticnet Wines Model
We can deploy an MLFlow model uploaded to an object store by using the MLFlow model server implementation as the config below:

In [10]:
%%writefile ../servers/mlflowserver/samples/elasticnet_wine.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: mlflow
spec:
  name: wines
  predictors:
  - componentSpecs:
    - spec:
        # We are setting high failureThreshold as installing conda dependencies
        # can take long time and we want to avoid k8s killing the container prematurely
        containers:
        - name: classifier
          livenessProbe:
            initialDelaySeconds: 80
            failureThreshold: 200
            periodSeconds: 5
            successThreshold: 1
            httpGet:
              path: /health/ping
              port: http
              scheme: HTTP
          readinessProbe:
            initialDelaySeconds: 80
            failureThreshold: 200
            periodSeconds: 5
            successThreshold: 1
            httpGet:
              path: /health/ping
              port: http
              scheme: HTTP
    graph:
      children: []
      implementation: MLFLOW_SERVER
      modelUri: gs://seldon-models/mlflow/elasticnet_wine
      name: classifier
    name: default
    replicas: 1

Overwriting ../servers/mlflowserver/samples/elasticnet_wine.yaml


In [ ]:
!kubectl apply -f ../servers/mlflowserver/samples/elasticnet_wine.yaml

In [ ]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=mlflow -o jsonpath='{.items[0].metadata.name}')

In [ ]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mlflow",namespace="seldon")

In [ ]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,11))
print(r)
assert(r.success==True)

In [ ]:
!kubectl delete -f ../servers/mlflowserver/samples/elasticnet_wine.yaml